In [1]:
in_dir  = r"C:\Users\GAMMA\Desktop\Johanna\MyoMet\DICOMs\MyoMet-CMR-025_033Y" #input directory
out_dir = r"C:\Users\GAMMA\Desktop\Johanna\MyoMet\Resclice" #output directory
#the slice thickness that should be regarded:
#None = thickness of 2D data
#0 = thickness of 3D data
#any number > 0 -> specific thickness
evaluate_slice_thickness = None

print(in_dir)
print(out_dir)


import pydicom as pd
import numpy as np
import scipy.interpolate
import os
import datetime
from ipywidgets import FloatProgress
from IPython.display import display
import warnings
 
warnings.filterwarnings('ignore')
print("### START OF RESLICING ###\n")

dir_sep = "\\" if  os.name == "nt" else "/"

#CREATE OUTPUT DIR
sub_dir = in_dir[in_dir.rfind(dir_sep)+1:]
out_dir = out_dir + dir_sep + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_rs3dt2d_" + sub_dir
os.mkdir(out_dir)
print("# output direcrtory created\n# search and load 3D data")

#PROGRESS BAR
file_num = sum([len(files) for r, d, files in os.walk(in_dir)])
progress = FloatProgress(min=0, max=file_num) # instantiate the bar
display(progress) # display the bar
progress.value = 0

#READ IN 3D DATA
data_3D = []
data_3D_value = np.array([])
data_3D_window_center = None
data_3D_window_width = None
data_3D_slice_thickness = 0
data_3D_z_resolution = 0
data_3D_x = np.array([])
data_3D_y = np.array([])
data_3D_z = np.array([])

for (dir_path, dir_names, file_names) in os.walk(in_dir):
    for file in file_names:
        progress.value += 1
        read = pd.dcmread(os.path.join(dir_path, file), force=True)
        
        try:
            print(read[0x0018, 0x0023].value, read.SeriesDescription)
            if not read[0x0018, 0x0023].value == "3D": # MR Acquisition Type
                break

            if not (read.SeriesDescription.endswith('_F') and 'Dixon' in  read.SeriesDescription): break
                
            data_3D.append(read)

            if data_3D_value.shape[0] == 0:            
                data_3D_value = np.asarray([read.pixel_array])
                
                if [0x0028, 0x1050] in read:
                    data_3D_window_center = int(read[0x0028, 0x1050].value)
                if [0x0028, 0x1051] in read:
                    data_3D_window_width = int(read[0x0028, 0x1051].value)
                
                data_3D_slice_thickness = np.asarray(read[0x0018, 0x0050].value, np.double)
                
                data_3D_x = np.arange(0, read.pixel_array.shape[1], 1) * read.PixelSpacing[1] + read.ImagePositionPatient[0]
                data_3D_y = np.arange(0, read.pixel_array.shape[0], 1) * read.PixelSpacing[0] + read.ImagePositionPatient[1]
                data_3D_z = np.asarray([read.ImagePositionPatient[2]])            
            else:
                data_3D_value = np.append(data_3D_value, np.asarray([read.pixel_array]), axis=0)
                data_3D_z = np.append(data_3D_z, np.asarray([read.ImagePositionPatient[2]]))
        except KeyError:
            pass
    
    if len(data_3D) != 0:
        sorted_index = np.argsort(data_3D_z)
        data_3D_z = data_3D_z[sorted_index]
        data_3D_value = data_3D_value[sorted_index, :, :]
        data_3D_z_resolution = abs(data_3D_z[1] - data_3D_z[0])
        break

print('Length of 3D Data: ', len(data_3D))

progress.value = file_num

 
 
#CHECK IF 3D DATA EXISTS
if len(data_3D) == 0:
    print("# no 3D data was found\n# abortion ----------------------")
else:
    print("# 3D data found and loaded\n# load 2D data and reslice")

    #PROGRESS BAR
    progress = FloatProgress(min=0, max=file_num) # instantiate the bar
    display(progress) # display the bar
    progress.value = 0
    
    studyinstanceuid = pd.uid.generate_uid()

    #READ IN 2D DATA AND WRITE RESCLICED 3D DATA
    for (dir_path, dir_names, file_names) in os.walk(in_dir):
        for file in file_names:
            #read dicom and check if it is not 3D
            read = pd.dcmread(os.path.join(dir_path, file), force=True)
            try:
                if not read[0x0018, 0x0023].value == "3D":
                    #if not "retro_iPAT" in read[0x0008, 0x103e].value:
                    if True:
                        # read in relevant dicom tags
                        image_position = np.asarray(read[0x0020, 0x0032].value, np.double)
                        direction_cosine = np.asarray(read[0x0020, 0x0037].value, np.double)
                        pixel_spacing = np.asarray(read[0x0028, 0x0030].value, np.double)

                        if evaluate_slice_thickness is None:
                            slice_thickness = np.asarray(read[0x0018, 0x0050].value, np.double)
                        elif evaluate_slice_thickness == 0:
                            slice_thickness = data_3D_slice_thickness
                        elif evaluate_slice_thickness > 0:
                            slice_thickness = evaluate_slice_thickness
                        else:
                            raise ValueError("The given evaluate_slice_thickness is neither None, 0 nor >0. Abortion.")

                        # create the matrix to transform indices of slice to coordinates
                        matrix = np.zeros((3, 3))
                        matrix[0, 0] = direction_cosine[0]
                        matrix[1, 0] = direction_cosine[1]
                        matrix[2, 0] = direction_cosine[2]
                        matrix[0, 1] = direction_cosine[3]
                        matrix[1, 1] = direction_cosine[4]
                        matrix[2, 1] = direction_cosine[5]
                        # for z-direction
                        #matrix[0:3, 2] = np.cross(direction_cosine[0:3], direction_cosine[3:]) / np.linalg.norm(np.cross(direction_cosine[0:3], direction_cosine[3:]))

                        # all x, y combinations of slice
                        x = np.arange(0, read.pixel_array.shape[1], 1)
                        y = np.arange(0, read.pixel_array.shape[0], 1)
                        X, Y = np.meshgrid(x,y)
                        x = np.reshape(X, -1)
                        y = np.reshape(Y, -1)

                        # vector for matrix vector product to transform indices of slice to coordinates
                        vector = np.zeros((x.shape[0], 3))
                        vector[:, 0] = x * pixel_spacing[1]
                        vector[:, 1] = y * pixel_spacing[0]

                        # transformation
                        product = np.transpose(np.dot(matrix, np.transpose(vector)))  # x y z
                        product = np.add(product[:, 0:3], image_position.reshape((1, 3)))

                        # evaluate the orthogonal normal of the slice
                        v1 = product[1, :] - product[0, :]
                        v2 = product[read.pixel_array.shape[0], :] - product[0, :]
                        n = np.cross(v1, v2)
                        n = n / np.linalg.norm(n)
                        
                        # number of points above and under the evaluation point along the normal
                        num = int(round(slice_thickness / (2*data_3D_z_resolution)))
                        # distance of the points (close to 3D resolution)
                        if num > 0:
                            dist = slice_thickness / (2*num)
                        else:
                            dist = 0

                        # creates shifted slices, evaluates the interpolated values and weights them (here all the same weight = average)
                        interpolate = np.zeros(product.shape[0])
                        for i in range(-num, num+1):
                            slice_2D = np.copy(product) + (n * dist * i)
                            slice_2D[:,[0,1,2]] = slice_2D[:,[2,1,0]]
                            interpolate = interpolate + scipy.interpolate.interpn((data_3D_z,data_3D_y,data_3D_x), data_3D_value, slice_2D, bounds_error = False, fill_value = 0)
                        interpolate = interpolate / (2*num+1)
                        
                        
                        # writes back the interpolated values into the Pixel Data of the dicom
                        new_pixel_array = np.copy(read.pixel_array)
                        new_pixel_array[y,x] = interpolate

                        # change pixel array and corresponding slice thickness
                        read.PixelData = new_pixel_array.tobytes()
                        read[0x0018, 0x0050].value = str(slice_thickness)

                        if not data_3D_window_center is None:
                            if [0x0028, 0x1050] in read:
                                read[0x0028, 0x1050].value = str(data_3D_window_center)
                            else:
                                read.add_new([0x0028, 0x1050], "DS", str(data_3D_window_center))

                        if not data_3D_window_width is None:
                            if [0x0028, 0x1051] in read:
                                read[0x0028, 0x1051].value = str(data_3D_window_width)
                            else:
                                read.add_new([0x0028, 0x1051], "DS", str(data_3D_window_width))
                    else:
                        pass
                    
                    # change some tags to make clear it is resliced   
                    read[0x0008, 0x1030].value = "Reslicing 3D to 2D"
                    read[0x0008, 0x0020].value = "19000101"
                    read[0x0008, 0x0021].value = "19000101"
                    read[0x0008, 0x0022].value = "19000101"
                    read[0x0008, 0x0023].value = "19000101"
                    read[0x0008, 0x103E].value = "rs3dt2d_" + read[0x0008, 0x103E].value
                    ##########
                    ## TODO ##
                    ##################################################
                    ## Overwrite study instance uid: 0x0020, 0x000D ##
                    ## Overwrite patient name      : 0x0010, 0x1001 ##
                    ##################################################
                    read[0x0020, 0x000D].value = studyinstanceuid
                    
                    # save the dicom as new one in the output dir with the same subdir structure
                    sub_dir = dir_path[dir_path.rfind(dir_sep):]
                    if not os.path.isdir(out_dir + sub_dir):
                        os.mkdir(out_dir + sub_dir)
                    read.save_as(out_dir + sub_dir + dir_sep + file)
            except KeyError:
                print("cannot handle file: " + os.path.join(dir_path, file))
            progress.value = progress.value + 1
            
    progress.value = file_num
    print("### END OF RESLICING ###")

 


C:\Users\GAMMA\Desktop\Johanna\MyoMet\DICOMs\MyoMet-CMR-025_033Y
C:\Users\GAMMA\Desktop\Johanna\MyoMet\Resclice
### START OF RESLICING ###

# output direcrtory created
# search and load 3D data


FloatProgress(value=0.0, max=1584.0)

2D trufi_loc_multi_iPAT_neu
2D trufi_loc_multi_iPAT iso
2D trufi_2-chamber_iPAT
2D trufi_4-chamber_iPAT
2D trufi_shortaxis_iPAT
2D T2_HASTE_db_nbh
2D t2_trufi_axial_ local_bh
2D t2_trufi_coronal_ local_bh
2D cine_tf2d12_retro_iPAT_4 cv
2D cine_tf2d12_retro_iPAT_3 cv
2D cine_tf2d12_retro_iPAT_2 cv
2D cine_tf2d12_retro_iPAT_RV
2D cine_tf2d12_retro_iPAT_4 cv
2D cine_tf2d12_retro_iPAT_4 cv
2D pre_MOLLI_5(3)3_4cv 1041B
2D pre_MOLLI_5(3)3_4cv 1041B_MOCO
2D pre_MOLLI_5(3)3_4cv 1041B_MOCO_PSIR
2D pre_MOLLI_5(3)3_4cv 1041B_MOCO_T1S
2D pre_MOLLI_5(3)3_4cv 1041B_MOCO_ERR
2D pre_MOLLI_5(3)3_4cv 1041B_MOCO_T1
2D pre_MOLLI_5(3)3_sax bas 1041B
2D pre_MOLLI_5(3)3_sax bas 1041B_MOCO
2D pre_MOLLI_5(3)3_sax bas 1041B_MOCO_PSIR
2D pre_MOLLI_5(3)3_sax bas 1041B_MOCO_T1S
2D pre_MOLLI_5(3)3_sax bas 1041B_MOCO_ERR
2D pre_MOLLI_5(3)3_sax bas 1041B_MOCO_T1
2D pre_MOLLI_5(3)3_sax mitt 1041B
2D pre_MOLLI_5(3)3_sax mitt 1041B_MOCO
2D pre_MOLLI_5(3)3_sax mitt 1041B_MOCO_PSIR
2D pre_MOLLI_5(3)3_sax mitt 1041B_MOCO_T

3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_CASPR_Dixon_TRA_Acc6.4_GMDMocoCGSense_F
3D 1111_GRE_iiNav_T2prep_

FloatProgress(value=0.0, max=1584.0)

cannot handle file: C:\Users\GAMMA\Desktop\Johanna\MyoMet\DICOMs\MyoMet-CMR-025_033Y\series1001-unknown\img0001-unknown.dcm
### END OF RESLICING ###


In [ ]:
 
import sys
import pydicom as pd
import numpy as np
import scipy

print("Package\t\tVersion\n------------------------")
print("Python\t\t" + sys.version)
print("Pydicom\t\t" + pd.__version__)
print("NumPy\t\t" + np.__version__)
print ("SciPy\t\t" + scipy.__version__)
